In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('user_comments_200k.csv')

In [3]:
all_usernames = list(set(df.username))

In [4]:
all_tweets = list(df.comment)

In [5]:
NUM_USERS = 400
NUM_FOLLOWS_PER_USER = 399
NUM_TWEETS_PER_USER = 150
MAX_TWEET_LEN = 100
SQLITE_FILEPATH = 'lab/benchmark.sqlite'

In [6]:
import os

os.rename(SQLITE_FILEPATH, SQLITE_FILEPATH + '.bak')

In [7]:
import sqlite3

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(SQLITE_FILEPATH)

# Create a cursor object
c = conn.cursor()

# Create tables
c.execute('''
create table users (
  id integer primary key,
  username text unique
);
''')

c.execute('''
create table tweets (
  id integer primary key,
  poster_id integer references users(id),
  content text,
  post_time integer
);
''')

c.execute('''
create table follows (
  follower_id integer references users(id),
  followee_id integer references users(id)
);
''')

c.execute('''
create table timelines (
  username text unique references users(username), 
  timeline_json text
);
''')

# Commit the changes
conn.commit()

In [8]:
usernames = all_usernames[:NUM_USERS]

In [9]:
# Insert usernames into the table
for username in usernames:
    c.execute(
'''
INSERT INTO users (username) VALUES (?);
''',
        (username,)
    )
        
conn.commit()

In [10]:
# Execute the SELECT query
c.execute("SELECT DISTINCT id FROM users")

# Fetch all the rows
rows = c.fetchall()

user_ids = []
for row in rows:
    user_ids.append(row[0])

In [11]:
import random

for user_id in user_ids:
    follower_ids = random.sample(user_ids, k=NUM_FOLLOWS_PER_USER)
    for follower_id in follower_ids:
        c.execute(
'''
INSERT INTO follows (follower_id, followee_id) VALUES (?, ?);
''',
        (follower_id, user_id)
)
        
conn.commit()

In [12]:
for user_id in user_ids:
    for _ in range(NUM_TWEETS_PER_USER):
        c.execute(
'''
INSERT INTO tweets (poster_id, content, post_time) VALUES (?, ?, abs(random() % 1680750000))
''',
            (user_id, random.choice(all_tweets)[:MAX_TWEET_LEN])
)
    
    conn.commit()

In [13]:
# This is run within the lab to keep the db/image smaller
# c.execute(
# '''
# insert into timelines
# select
#  users.username,
#  json_group_array(json_object(
#   'tweet_id', tweets.id, 
#   'poster_id', tweets.poster_id, 
#   'content', tweets.content, 
#   'post_time', tweets.post_time))
# from tweets
# join follows on follows.followee_id = tweets.poster_id
# join users on users.id = follows.follower_id
# group by users.username;
# '''
# )
    
# conn.commit()

In [14]:
# Close the connection
conn.close()